In [12]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

In [13]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
from sklearn import preprocessing
import pickle

In [ ]:
searched_date='original'

In [14]:
#元データの読み込み
spec=pd.read_csv('preprocessed_spec_iqr1_with_bit_{}.csv'.format(searched_date))
spec=spec.drop(spec.columns[0],axis=1)

In [15]:
#予測結果の読み込み
prediction=pd.read_csv('intermediate_data/prediction_xgboost_{}.csv'.format(searched_date))
prediction=prediction.set_index('index')

In [16]:
prediction_post=prediction.copy()
prediction_post['dif']=prediction['price']-prediction['prediction']
prediction_post['dif_per']=np.round((abs(prediction['prediction']-prediction['price'])/prediction['price'])*100,2)

In [17]:
#誤差が5%以内の物件の割合
error5=np.round((prediction_post[prediction_post['dif_per']<5].shape[0]/prediction_post.shape[0])*100,2)
print('誤差が5%以内の物件の割合:{}%'.format(error5))

誤差が5%以内の物件の割合:99.46%


In [18]:
#LabelEncoderの読み込み
with open('label/label_encoder_{}.pickle'.format(searched_date),'rb') as f:
    labels=pickle.load(f)

In [19]:
for key in labels.keys():
    spec[key]=labels[key].inverse_transform(spec[key].astype(int))

/Users/toshio/py3env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/toshio/py3env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/toshio/py3env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/toshio/py3env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

In [20]:
final=prediction_post.loc[:,['prediction','dif','dif_per']].join(spec)
final=final.sort_index()

In [21]:
#誤差が5%以上の物件の表示
final.loc[prediction_post['dif_per']>5]

,prediction,dif,dif_per,price,詳細情報,平米単価,専有面積,間取り,築年数,構造,階数,向き,敷地の権利形態,管理費等,修繕積立金,取引態様,最寄り駅,駅からの手段,駅からの時間
index,,,,,,,,,,,,,,,,,,,
480,414.984924,-44.984924,12.16,370.0,13762395.0,142300.0,26.01,2K,38.0,鉄骨造,2.0,南,所有権,6000.0,4000.0,売主,東村山,徒歩,15.0
845,7162.137695,-5262.137695,276.95,1900.0,15057832.0,1798400.0,60.61,2LDK,1.0,鉄筋コンクリート（ＲＣ）,4.0,北西,所有権,20956.0,6061.0,仲介,目黒,徒歩,2.0
1281,1120.175659,59.824341,5.07,1180.0,12030826.0,595400.0,19.82,ワンルーム,34.0,鉄筋コンクリート（ＲＣ）,3.0,西,所有権,7600.0,2260.0,仲介,経堂,徒歩,6.0
1557,8661.063477,1188.936523,12.07,9850.0,14536357.0,1310600.0,75.16,1LDK,17.0,鉄筋コンクリート（ＲＣ）,5.0,南西,所有権,18010.0,10960.0,仲介,表参道,徒歩,13.0
1572,1586.361328,93.638672,5.57,1680.0,15008590.0,185100.0,90.79,4LDK,32.0,鉄筋コンクリート（ＲＣ）,5.0,南,所有権,6000.0,9300.0,仲介,京王多摩センター,徒歩,17.0
1616,330.566620,19.433380,5.55,350.0,14392052.0,219500.0,15.95,ワンルーム,27.0,鉄筋コンクリート（ＲＣ）,3.0,東,所有権,5500.0,2000.0,仲介,豊田,徒歩,16.0
1867,9799.509766,-8399.509766,599.96,1400.0,14915786.0,743900.0,139.81,2SLDK,30.0,鉄骨鉄筋コンクリート（ＳＲＣ）,12.0,南,所有権,44900.0,12220.0,仲介,若松河田,徒歩,3.0
1951,1951.597290,-161.597290,9.03,1790.0,14062092.0,1216900.0,14.71,ワンルーム,39.0,鉄筋コンクリート（ＲＣ）,6.0,東,所有権,5860.0,5260.0,仲介,大門,徒歩,1.0
2000,921.418823,63.581177,6.45,985.0,8805034.0,486200.0,20.26,1K,30.0,鉄骨鉄筋コンクリート（ＳＲＣ）,1.0,南西,所有権,5500.0,5630.0,仲介,平和台,徒歩,8.0


In [22]:
#割安物件の表示
final.loc[(prediction_post['dif_per']>5)&((prediction_post['dif']<0))]

,prediction,dif,dif_per,price,詳細情報,平米単価,専有面積,間取り,築年数,構造,階数,向き,敷地の権利形態,管理費等,修繕積立金,取引態様,最寄り駅,駅からの手段,駅からの時間
index,,,,,,,,,,,,,,,,,,,
480,414.984924,-44.984924,12.16,370.0,13762395.0,142300.0,26.01,2K,38.0,鉄骨造,2.0,南,所有権,6000.0,4000.0,売主,東村山,徒歩,15.0
845,7162.137695,-5262.137695,276.95,1900.0,15057832.0,1798400.0,60.61,2LDK,1.0,鉄筋コンクリート（ＲＣ）,4.0,北西,所有権,20956.0,6061.0,仲介,目黒,徒歩,2.0
1867,9799.509766,-8399.509766,599.96,1400.0,14915786.0,743900.0,139.81,2SLDK,30.0,鉄骨鉄筋コンクリート（ＳＲＣ）,12.0,南,所有権,44900.0,12220.0,仲介,若松河田,徒歩,3.0
1951,1951.597290,-161.597290,9.03,1790.0,14062092.0,1216900.0,14.71,ワンルーム,39.0,鉄筋コンクリート（ＲＣ）,6.0,東,所有権,5860.0,5260.0,仲介,大門,徒歩,1.0
2007,3627.341797,-447.341797,14.07,3180.0,14930373.0,3638500.0,8.74,2LDK,47.0,鉄骨鉄筋コンクリート（ＳＲＣ）,8.0,南,所有権,7500.0,11000.0,仲介,東中野,徒歩,14.0
2159,381.508881,-23.508881,6.57,358.0,11283026.0,237500.0,15.08,ワンルーム,37.0,鉄骨造,2.0,北東,所有権,4000.0,3500.0,代理,八王子,徒歩,13.0
2585,1332.250977,-82.250977,6.58,1250.0,15217647.0,1511800.0,67.80,2LDK,10.0,鉄筋コンクリート（ＲＣ）,3.0,南西,所有権,21800.0,4750.0,仲介,四ツ谷,徒歩,4.0
3658,136.395050,-11.395050,9.12,125.0,11799673.0,34200.0,36.61,1DK,40.0,鉄筋コンクリート（ＲＣ）,3.0,南,旧法地上,0.0,14400.0,仲介,武蔵小金井,徒歩,19.0
3866,6587.495117,-607.495117,10.16,5980.0,12893025.0,865600.0,69.09,2LDK,10.0,鉄筋コンクリート（ＲＣ）,1.0,南,所有権,33474.0,14900.0,仲介,神楽坂,徒歩,6.0
